In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.layers import Dropout 
from keras import backend as K
import json
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import shuffle
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
# Swish activation fucntion
def swish(x, beta=1.0):
    return x * K.sigmoid(beta * x)

In [3]:
# Learning rate scheduler
def scheduler(epoch):
    if epoch < 2:
        return 0.001
    else:
        return 0.001 * 0.98

In [4]:
# Load the data, create the one-hot labels, and prepare the K-Fold Cross Validation
CWD = os.getcwd()
string = os.path.join(CWD, 'data/put.mat')
all_data = loadmat(string) 
data_steps = all_data['features']
data_steps_array = np.array(data_steps)
data_labels = all_data['labels']
data_labels_arr = np.array(data_labels)
data_labels_arr = np.reshape(data_labels_arr, (data_labels_arr.shape[1]))

num_classes = 6

data_labels_array = np.zeros((data_labels_arr.shape[0], 6))
for i in range(data_labels_arr.shape[0]):
    for j in range(6):
        if (data_labels_arr[i] == j):
            data_labels_array[i,j] = 1.0 

# normalization of the data
normed = preprocessing.scale(data_steps_array)    

normed, data_labels_array = shuffle(normed, data_labels_array, random_state=47)

# K-Fold Cross validation preparation
n_split = 10 # 10-fold cross validation
train_index = []
test_index = []
for train_ind,test_ind in KFold(n_split, random_state=47).split(normed):
    train_index.append(train_ind)
    test_index.append(test_ind)

In [5]:
# train the Fully Connected Neurons for Frequency Features based on paper:
# "What am I touching? Learning to classify terrain via haptic sensing"
num_epochs = 1000
batch_size_num = 64
val_acc = {'{:1d}'.format(j):[] for j in range(n_split)}

for i in range(n_split):
    # Create the model with keras
    model  = tf.keras.Sequential([   

                    tf.keras.layers.Dense(256, activation = swish),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Dropout(0.2, seed=47),
                    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = swish),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Dropout(0.4, seed=47),
                    tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = swish),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Dropout(0.4, seed=47),
                    tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = swish),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Dropout(0.4, seed=47),
                    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = swish),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Dropout(0.4, seed=47),
                    tf.keras.layers.Dense(6, kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation = 'softmax')
    ])
  
    callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])    
    x_train,x_test=normed[train_index[i]],normed[test_index[i]]
    y_train,y_test=data_labels_array[train_index[i]],data_labels_array[test_index[i]]
    history = model.fit(x_train,y_train, batch_size=batch_size_num, epochs = num_epochs, 
                        callbacks=[callback], validation_data = (x_test,y_test))
    print(100*'*', '  ', i)
    val_acc['{:1d}'.format(i)] = history.history['val_acc']

Train on 8391 samples, validate on 933 samples
Epoch 1/1000
8391/8391 [==============================] - 2s 180us/step - loss: 1.4168 - acc: 0.4401 - val_loss: 1.2677 - val_acc: 0.4984
Epoch 2/1000
8391/8391 [==============================] - 1s 83us/step - loss: 1.1344 - acc: 0.5679 - val_loss: 1.0344 - val_acc: 0.6109
Epoch 3/1000
8391/8391 [==============================] - 1s 83us/step - loss: 0.9693 - acc: 0.6447 - val_loss: 0.9227 - val_acc: 0.6527
Epoch 4/1000
8391/8391 [==============================] - 1s 87us/step - loss: 0.8900 - acc: 0.6756 - val_loss: 0.8948 - val_acc: 0.6613
Epoch 5/1000
8391/8391 [==============================] - 1s 85us/step - loss: 0.8162 - acc: 0.7092 - val_loss: 0.8273 - val_acc: 0.7020
Epoch 6/1000
8391/8391 [==============================] - 1s 87us/step - loss: 0.7241 - acc: 0.7499 - val_loss: 0.7704 - val_acc: 0.7160
Epoch 7/1000
8391/8391 [==============================] - 1s 87us/step - loss: 0.6721 - acc: 0.7663 - val_loss: 0.7805 - val_acc: 

8391/8391 [==============================] - 1s 97us/step - loss: 0.1863 - acc: 0.9715 - val_loss: 1.0304 - val_acc: 0.7771
Epoch 61/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.1640 - acc: 0.9826 - val_loss: 1.0277 - val_acc: 0.7792
Epoch 62/1000
8391/8391 [==============================] - 1s 99us/step - loss: 0.2326 - acc: 0.9603 - val_loss: 1.0660 - val_acc: 0.7717
Epoch 63/1000
8391/8391 [==============================] - 1s 99us/step - loss: 0.1544 - acc: 0.9867 - val_loss: 1.1581 - val_acc: 0.7578
Epoch 64/1000
8391/8391 [==============================] - 1s 100us/step - loss: 0.1367 - acc: 0.9911 - val_loss: 1.1626 - val_acc: 0.7738
Epoch 65/1000
8391/8391 [==============================] - 1s 99us/step - loss: 0.1300 - acc: 0.9933 - val_loss: 1.1667 - val_acc: 0.7803
Epoch 66/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.1182 - acc: 0.9964 - val_loss: 1.1738 - val_acc: 0.7824
Epoch 67/1000
8391/8391 [======================

8391/8391 [==============================] - 1s 97us/step - loss: 0.1406 - acc: 0.9836 - val_loss: 1.0742 - val_acc: 0.7653
Epoch 120/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.1038 - acc: 0.9933 - val_loss: 1.1204 - val_acc: 0.7610
Epoch 121/1000
8391/8391 [==============================] - 1s 95us/step - loss: 0.0916 - acc: 0.9965 - val_loss: 1.1464 - val_acc: 0.7706
Epoch 122/1000
8391/8391 [==============================] - 1s 97us/step - loss: 0.0872 - acc: 0.9968 - val_loss: 1.0693 - val_acc: 0.7814
Epoch 123/1000
8391/8391 [==============================] - 1s 96us/step - loss: 0.0841 - acc: 0.9976 - val_loss: 1.1743 - val_acc: 0.7781
Epoch 124/1000
8391/8391 [==============================] - 1s 96us/step - loss: 0.1044 - acc: 0.9909 - val_loss: 1.2333 - val_acc: 0.7653
Epoch 125/1000
8391/8391 [==============================] - 1s 97us/step - loss: 0.1655 - acc: 0.9695 - val_loss: 1.1611 - val_acc: 0.7513
Epoch 126/1000
8391/8391 [================

8391/8391 [==============================] - 1s 97us/step - loss: 0.0501 - acc: 1.0000 - val_loss: 1.2048 - val_acc: 0.7856
Epoch 179/1000
8391/8391 [==============================] - 1s 96us/step - loss: 0.0488 - acc: 1.0000 - val_loss: 1.2014 - val_acc: 0.7835
Epoch 180/1000
8391/8391 [==============================] - 1s 96us/step - loss: 0.0475 - acc: 1.0000 - val_loss: 1.2067 - val_acc: 0.7867
Epoch 181/1000
8391/8391 [==============================] - 1s 95us/step - loss: 0.0462 - acc: 1.0000 - val_loss: 1.2042 - val_acc: 0.7835
Epoch 182/1000
8391/8391 [==============================] - 1s 99us/step - loss: 0.0450 - acc: 1.0000 - val_loss: 1.2064 - val_acc: 0.7824
Epoch 183/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.0438 - acc: 1.0000 - val_loss: 1.2171 - val_acc: 0.7846
Epoch 184/1000
8391/8391 [==============================] - 1s 100us/step - loss: 0.0427 - acc: 1.0000 - val_loss: 1.2232 - val_acc: 0.7824
Epoch 185/1000
8391/8391 [===============

8391/8391 [==============================] - 1s 97us/step - loss: 0.0592 - acc: 0.9932 - val_loss: 1.7241 - val_acc: 0.6785
Epoch 238/1000
8391/8391 [==============================] - 1s 97us/step - loss: 0.4061 - acc: 0.8790 - val_loss: 0.8219 - val_acc: 0.7567
Epoch 239/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.1362 - acc: 0.9720 - val_loss: 1.0081 - val_acc: 0.7663
Epoch 240/1000
8391/8391 [==============================] - 1s 97us/step - loss: 0.0904 - acc: 0.9870 - val_loss: 1.1203 - val_acc: 0.7674
Epoch 241/1000
8391/8391 [==============================] - 1s 98us/step - loss: 0.0635 - acc: 0.9976 - val_loss: 1.1626 - val_acc: 0.7771
Epoch 242/1000
8391/8391 [==============================] - 1s 97us/step - loss: 0.0559 - acc: 0.9987 - val_loss: 1.2205 - val_acc: 0.7706
Epoch 243/1000
8391/8391 [==============================] - 1s 99us/step - loss: 0.0725 - acc: 0.9930 - val_loss: 1.2293 - val_acc: 0.7674
Epoch 244/1000
8391/8391 [================

...

8392/8392 [==============================] - 1s 122us/step - loss: 0.0142 - acc: 1.0000 - val_loss: 1.6496 - val_acc: 0.7597
Epoch 722/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0139 - acc: 1.0000 - val_loss: 1.6912 - val_acc: 0.7629
Epoch 723/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.3276 - acc: 0.9026 - val_loss: 0.9477 - val_acc: 0.7232
Epoch 724/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.1830 - acc: 0.9416 - val_loss: 1.0704 - val_acc: 0.7361
Epoch 725/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0858 - acc: 0.9790 - val_loss: 1.2288 - val_acc: 0.7382
Epoch 726/1000
8392/8392 [==============================] - 1s 118us/step - loss: 0.0436 - acc: 0.9934 - val_loss: 1.3504 - val_acc: 0.7554
Epoch 727/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0347 - acc: 0.9967 - val_loss: 1.4530 - val_acc: 0.7564
Epoch 728/1000
8392/8392 [=========

8392/8392 [==============================] - 1s 119us/step - loss: 0.0191 - acc: 1.0000 - val_loss: 1.5904 - val_acc: 0.7736
Epoch 780/1000
8392/8392 [==============================] - 1s 121us/step - loss: 0.0189 - acc: 1.0000 - val_loss: 1.5903 - val_acc: 0.7736
Epoch 781/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0186 - acc: 1.0000 - val_loss: 1.5897 - val_acc: 0.7725
Epoch 782/1000
8392/8392 [==============================] - 1s 122us/step - loss: 0.0184 - acc: 1.0000 - val_loss: 1.5894 - val_acc: 0.7704
Epoch 783/1000
8392/8392 [==============================] - 1s 118us/step - loss: 0.0181 - acc: 1.0000 - val_loss: 1.5884 - val_acc: 0.7725
Epoch 784/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0178 - acc: 1.0000 - val_loss: 1.5875 - val_acc: 0.7736
Epoch 785/1000
8392/8392 [==============================] - 1s 122us/step - loss: 0.0176 - acc: 1.0000 - val_loss: 1.5864 - val_acc: 0.7758
Epoch 786/1000
8392/8392 [=========

8392/8392 [==============================] - 1s 110us/step - loss: 0.0205 - acc: 1.0000 - val_loss: 1.6654 - val_acc: 0.7672
Epoch 838/1000
8392/8392 [==============================] - 1s 111us/step - loss: 0.0203 - acc: 1.0000 - val_loss: 1.6683 - val_acc: 0.7661
Epoch 839/1000
8392/8392 [==============================] - 1s 110us/step - loss: 0.0201 - acc: 1.0000 - val_loss: 1.6711 - val_acc: 0.7650
Epoch 840/1000
8392/8392 [==============================] - 1s 102us/step - loss: 0.0199 - acc: 1.0000 - val_loss: 1.6716 - val_acc: 0.7672
Epoch 841/1000
8392/8392 [==============================] - 1s 99us/step - loss: 0.0197 - acc: 1.0000 - val_loss: 1.6719 - val_acc: 0.7672
Epoch 842/1000
8392/8392 [==============================] - 1s 99us/step - loss: 0.0195 - acc: 1.0000 - val_loss: 1.6718 - val_acc: 0.7672
Epoch 843/1000
8392/8392 [==============================] - 1s 99us/step - loss: 0.0192 - acc: 1.0000 - val_loss: 1.6702 - val_acc: 0.7672
Epoch 844/1000
8392/8392 [============

Epoch 896/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0128 - acc: 1.0000 - val_loss: 1.6447 - val_acc: 0.7725
Epoch 897/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0125 - acc: 1.0000 - val_loss: 1.6438 - val_acc: 0.7758
Epoch 898/1000
8392/8392 [==============================] - 1s 121us/step - loss: 0.0123 - acc: 1.0000 - val_loss: 1.6548 - val_acc: 0.7736
Epoch 899/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0120 - acc: 1.0000 - val_loss: 1.6575 - val_acc: 0.7768
Epoch 900/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0118 - acc: 1.0000 - val_loss: 1.6707 - val_acc: 0.7790
Epoch 901/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0115 - acc: 1.0000 - val_loss: 1.6744 - val_acc: 0.7758
Epoch 902/1000
8392/8392 [==============================] - 1s 121us/step - loss: 0.0113 - acc: 1.0000 - val_loss: 1.6860 - val_acc: 0.7779
Epoch 903/1000
8392/

8392/8392 [==============================] - 1s 120us/step - loss: 0.0170 - acc: 1.0000 - val_loss: 1.6038 - val_acc: 0.7704
Epoch 955/1000
8392/8392 [==============================] - 1s 117us/step - loss: 0.0168 - acc: 1.0000 - val_loss: 1.6146 - val_acc: 0.7704
Epoch 956/1000
8392/8392 [==============================] - 1s 118us/step - loss: 0.0166 - acc: 1.0000 - val_loss: 1.6222 - val_acc: 0.7693
Epoch 957/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0164 - acc: 1.0000 - val_loss: 1.6290 - val_acc: 0.7704
Epoch 958/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0162 - acc: 1.0000 - val_loss: 1.6341 - val_acc: 0.7682
Epoch 959/1000
8392/8392 [==============================] - 1s 120us/step - loss: 0.0160 - acc: 1.0000 - val_loss: 1.6384 - val_acc: 0.7682
Epoch 960/1000
8392/8392 [==============================] - 1s 119us/step - loss: 0.0158 - acc: 1.0000 - val_loss: 1.6413 - val_acc: 0.7704
Epoch 961/1000
8392/8392 [=========

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             multiple                  14080     
_________________________________________________________________
batch_normalization_45 (Batc multiple                  1024      
_________________________________________________________________
dropout_45 (Dropout)         multiple                  0         
_________________________________________________________________
dense_55 (Dense)             multiple                  131584    
_________________________________________________________________
batch_normalization_46 (Batc multiple                  2048      
_________________________________________________________________
dropout_46 (Dropout)         multiple                  0         
_________________________________________________________________
dense_56 (Dense)             multiple                  131328    
__________

In [ ]:
# The final Report
maxim = [np.max(val_acc['{:1d}'.format(i)]) for i in range(10)]

print('mean accuracy: ', np.mean(maxim), ' std: ',  np.std(maxim), ' min: ', np.min(maxim), ' max: ', np.max(maxim))